In [ ]:
import math
import random
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit

In [ ]:
from pywalk import run_walk, get_arrival_list

In [ ]:
walk = run_walk(n_steps = 10000, alpha=2.)

In [ ]:
px.scatter(walk)

In [ ]:
walk = list(filter(lambda x: x<1e15, walk))

ranks = range(len(walk))

fig = px.scatter(x=range(len(walk)), y=[f/sum(walk) for f in sorted(walk, reverse=True)])
fig.add_scatter(x=list(range(1,1+len(walk))), y=[0.1/_x for _x in range(1,1+len(walk))])

fig.update_layout({"xaxis":{"type":"log"},
                   "yaxis":{"type":"log"}})

In [ ]:
bin_counts, bin_edges = np.histogram(walk, bins=np.logspace(0,4,100))

x = (bin_edges[1:]+bin_edges[:-1])/2.

func = lambda x, C: C*np.power(x,-1)
popt, pcov = curve_fit(func, x, bin_counts)

fig = px.scatter(x=x, y=bin_counts)
fig.add_scatter(x=x, y=func(x,*popt))

fig.update_layout({"xaxis":{"type":"log"},
                   "yaxis":{"type":"log"}})

In [ ]:
arrivals = get_arrival_list(n_stats = 10000, n_steps = 500, alpha=2.)


In [ ]:
from scipy.special import factorial

def rho_n(n:int, x, alpha = 2.):
    if n==0:
        return 1/200.
    else:
        return 1. / alpha * math.log(x/alpha**n)
    
def rho(x, n_iter = 45, *args, **kwargs):
    _rho = 1. - rho_n(n_iter,x)
    for n in list(range(0, n_iter))[::-1]:
        #print("1-rho_{}".format(n))
        _rho = 1.-rho_n(n, x, *args, **kwargs)*_rho
    return 1. + _rho

In [ ]:
fig = go.Figure(layout={"height":400, "width":800})

bin_counts, bin_edges = np.histogram(arrivals, bins=np.logspace(0,5, 100), density=True)

x = (bin_edges[1:]+bin_edges[:-1])/2.
fig.add_scatter(x=x, y=bin_counts, mode="markers", name="simulations")

x = np.logspace(0, 5, 41)

rhox = np.array([rho(_x, alpha=2.) for _x in x])
rhox = 1e5*rhox/sum(rhox)
mask = [rhox>0]
fig.add_scatter(x=x[mask], y=rhox[mask], mode="lines", name="conto numerico <br>ispirato a PRE Gherardi")


fig.update_layout({"xaxis":{"type":"log", "title":"x"},
                   "yaxis":{"type":"log", "title":"rho(x)"}})

fig.show()
fig.write_image("rhox_sim_num.pdf")